**Pre-Processing** 

Loop through the raw daily .csv files in the dbfs and align with the master event file so we have fish events aligned to the resistivity data. Save the labelled data to a new folder called labelled. 

Start of by aligning the raw resistivity with the fish events. 

In [0]:
# Get packages
import pandas as pd
import glob
import os

# Load and prepare events file with all of the known fish events
events_path = "/dbfs/mnt/lab/unrestricted/rachel.lennon@defra.gov.uk/Thr 20 process/test_KMThu16_2021_07_eventonly.csv"
events = pd.read_csv(events_path, header=0)

# Ensure datetime is timezone-aware
events['Time'] = pd.to_datetime(events['Time'], utc=True)

# Create +/- event windows 2.5 seconds before and 2.5 seconds after as per AF parameters - we might want to truncate this?
events['start_time'] = events['Time'] - pd.Timedelta(seconds=2.5)
events['end_time']   = events['Time'] + pd.Timedelta(seconds=2.5)


# Function to label fish presence from the timestamps in the raw df 
def label_fish_presence(df, events_df):
    df['Time'] = pd.to_datetime(df['Time'], utc=True)
    df['fish_present'] = 0
    
    for _, event in events_df.iterrows():
        mask = (df['Time'] >= event['start_time']) & (df['Time'] <= event['end_time'])
        df.loc[mask, 'fish_present'] = 1
    
    return df

# Loop through all raw daily datasets
input_pattern = "/dbfs/mnt/lab/unrestricted/rachel.lennon@defra.gov.uk/DuplicateFree/*_no_duplicate"  
output_folder = "/dbfs/mnt/lab/unrestricted/rachel.lennon@defra.gov.uk/labeled" # where to put the labelled dailies

os.makedirs(output_folder, exist_ok=True)

# Run labelling function on all of the daily datasets
for file_path in glob.glob(input_pattern):
    print(f"Processing {file_path}...")

    # Load and prep daily data
    df = pd.read_csv(file_path, header=0)
    df.columns = ["timestamp", "upstream", "downstream"]

    # Convert ms timestamp to datetime making sure it is timezone aware too
    df["Time"] = pd.to_datetime(df["timestamp"], unit="ms", utc=True)
    df = df.drop(columns=["timestamp"])

    # Add the differential conductance to the daily df 
    df["differential_conductance"] = (df["downstream"] - df["upstream"]) / 2

    # Label fish events using the function 
    labeled_df = label_fish_presence(df, events)

    # Save to new CSVs in the output folder
    filename = os.path.basename(file_path).replace("_no_duplicate", "_labelled")
    save_path = os.path.join(output_folder, filename)
    labeled_df.to_csv(save_path, index=False)

print("All datasets labelled and saved.")


Look at the df to make sure it is structured as expected.

In [0]:
import pandas as pd

df = pd.read_csv("/dbfs/mnt/lab/unrestricted/rachel.lennon@defra.gov.uk/labeled/test_KMThu16_2021_07_17_15_36_26Z_labelled")

display(df)

Databricks filtered table. Run in Databricks to view.

Check the average sampling rate.

In [0]:
# Path to files
input_pattern = "/dbfs/mnt/lab/unrestricted/rachel.lennon@defra.gov.uk/labeled/*Z_labelled"

for file_path in glob.glob(input_pattern):
    print(f"Processing {file_path}...")
    
    # Load CSV
    df = pd.read_csv(file_path)
    
    # Convert Time column to datetime
    df['Time'] = pd.to_datetime(df['Time'], format='ISO8601')
    
    # Sort by time
    df = df.sort_values('Time')
    
    # Calculate time differences in seconds
    df['delta_t'] = df['Time'].diff().dt.total_seconds()
    
    # Drop the first NaN
    delta_t = df['delta_t'].dropna()
    
    # Compute average sampling rate
    avg_sampling_rate = 1 / delta_t.mean()
    
    print(f"Average sampling rate for {file_path}: {avg_sampling_rate:.2f} Hz")




**Re-Sampling**

We have varying sample rate so we need to resample the data. Done using polyphase filtering with automatically derived FIR (Finite Impulse Response) filter to avoid ailising of the signal.

In [0]:
import pandas as pd
from fractions import Fraction
from scipy.signal import resample_poly
import numpy as np
import glob
import os

# Path to files
input_pattern = "/dbfs/mnt/lab/unrestricted/rachel.lennon@defra.gov.uk/labeled/*Z_labelled"
output_folder = "/dbfs/mnt/lab/unrestricted/rachel.lennon@defra.gov.uk/resampled" # where to put the resampled dailies

os.makedirs(output_folder, exist_ok=True)

for file_path in glob.glob(input_pattern):
    print(f"Resampling {file_path}...")
    
    # Load CSV
    df = pd.read_csv(file_path)
    df['Time'] = pd.to_datetime(df['Time'], format = "ISO8601")
    df = df.sort_values('Time')
    
    # Extract signal
    signal = df['differential_conductance'].values
    
    # Original sampling rate
    delta_t = df['Time'].diff().dt.total_seconds().dropna()
    fs_original = 1 / delta_t.mean()
    fs_target = 100
    
    # Compute up/down ratio
    ratio = Fraction(str(fs_target)) / Fraction(str(fs_original))
    ratio = ratio.limit_denominator()
    up, down = ratio.numerator, ratio.denominator
    
    # Resample signal
    signal_resampled = resample_poly(signal, up, down)
    
    # Build new uniform time index
    n_samples = len(signal_resampled)
    start_time = df['Time'].iloc[0]
    new_time_index = pd.date_range(start=start_time, periods=n_samples, freq=f"{1000/fs_target}ms")
    
    # Convert times to seconds since start for binning
    original_times_sec = (df['Time'] - start_time).dt.total_seconds()
    bin_edges_sec = np.append(np.arange(n_samples) / fs_target, n_samples / fs_target)
    
    # Get bin index for each original row
    bin_indices = np.digitize(original_times_sec, bin_edges_sec) - 1
    
    # Vectorized: group fish_present by bin index and take max (1 if any fish present)
    labels_resampled = (
    pd.Series(df['fish_present'].values)
    .groupby(bin_indices)
    .max()
    .reindex(range(n_samples), fill_value=0)
    .astype(int)
    .values
    )
    
    # Build resampled DataFrame
    df_resampled = pd.DataFrame({
    'Time': new_time_index,
    'differential_conductance': signal_resampled,
    'fish_present': labels_resampled
    })

        # Save to new CSVs in the output folder
    filename = os.path.basename(file_path).replace("_labelled", "_resampled")
    save_path = os.path.join(output_folder, filename)
    df_resampled.to_csv(save_path, index=False)

print("All datasets resampled and saved.")


**Bin into 5 second chunks**


We need to have windows of time to feed into the UNet, do this e = 1/5 Hz as an initial approach. 

In [0]:
import os
import pandas as pd
import numpy as np
from glob import glob
from sklearn.preprocessing import StandardScaler

# Parameters
input_folder = "/dbfs/mnt/lab/unrestricted/rachel.lennon@defra.gov.uk/resampled/*Z_resampled"  # folder with CSVs
output_folder = "/dbfs/mnt/lab/unrestricted/rachel.lennon@defra.gov.uk/cleaned/*Z_cleaned"
window_size = 600                  # 6 seconds per window
step_size = 500                    # 1 second stride 

# Make sure output folder exists
os.makedirs(output_folder, exist_ok=True)

# Function to scale a window
def scale_window(chunk):
    median = np.median(chunk)
    iqr = np.percentile(chunk, 75) - np.percentile(chunk, 25)
    if iqr == 0:
        return chunk - median  # avoid division by zero
    return (chunk - median) / iqr

# Process each CSV
all_windows = []

for csv_file in glob(os.path.join(input_folder)):
    df = pd.read_csv(csv_file)
    
    signal = df['differential_conductance'].to_numpy()
    fish_present = df['fish_present'].to_numpy()
    
    # Bin into windows
    for start in range(0, len(signal) - window_size + 1, step_size):
        end = start + window_size
        window = signal[start:end]
        label = 1 if fish_present[start:end].sum() > 0.5 else 0
        
        # Scale
        window_scaled = scale_window(window)
        
        # Append as dictionary
        all_windows.append({
            "signal_scaled": window_scaled.tolist(),
            "fish_present": label
        })


# Combine all windows into a single DataFrame
fish_clean = pd.DataFrame(all_windows)

# Save to cleaned folder
fish_clean.to_csv(os.path.join(output_folder, "fish_clean.csv"), index=False)


In [0]:
display(fish_clean)

Databricks filtered table. Run in Databricks to view.

**Balance training dataset**

The noise (0/no fish) to signal (1/fish) ratio is out of whack. We need to balance it so the ratio is more 1:1 and can be used as a training dataset. Otherwise it could label everything as 0 and we would still get 99% accuracy. 

In [0]:
# Get packages
import pandas as pd
import glob
import os

# Get the paths to the right folders
input_file = "/dbfs/mnt/lab/unrestricted/rachel.lennon@defra.gov.uk/cleaned/*Z_cleaned/fish_clean.csv" # Where the labelled daily datasets are
output_folder = "/dbfs/mnt/lab/unrestricted/rachel.lennon@defra.gov.uk/cleaned/" # Where to put the balanced df 

# Run through all of the labelled daily df and balance the noise to signal

df = pd.read_csv(input_file)
    
# Split fish vs no-fish
fish_df = df[df['fish_present'] == 1]
no_fish_df = df[df['fish_present'] == 0]
    
# Match counts so equalised
no_fish_sample = no_fish_df.sample(len(fish_df), random_state=42)
    
# Combine and shuffle
balanced_df = pd.concat([fish_df, no_fish_sample]).sample(frac=1, random_state=42)

# Save balanced CSV
balanced_df.to_csv(os.path.join(output_folder, "all_balanced.csv"), index=False)


